In [1]:
import cv2 as cv
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
original_rgb   = './dataset/original_data/rgb/'
original_depth   = './dataset/original_data/depth/'

target_rgb = './dataset/cropped/rgb/'
target_depth = './dataset/cropped/depth/'

original_csv_path = './dataset/original_data/data.csv' 
csv_file_path = './dataset/cropped/croped_data.csv'

original_data = pd.read_csv(original_csv_path)

cropped_data = []

for file_name in os.listdir(original_rgb):
    path_org_rgb = os.path.join(original_rgb, file_name)
    path_org_depth = os.path.join(original_depth, file_name)

    rgb_img = cv.imread(path_org_rgb)
    depth_img = cv.imread(path_org_depth)
    
    for ii in range(3):
        crop_size = np.random.randint(300,500)
        h, w, _ = rgb_img.shape
        
        rand_1 = np.random.randint(-50,50)
        rand_2 = np.random.randint(-50,50)
        
        crop_1 = crop_size + rand_1
        crop_2 = crop_size + rand_2
        
        rgb_cropped_img = rgb_img[(h-crop_1)//2:(h+crop_1)//2, (w-crop_2)//2-rand_1:(w+crop_2)//2+rand_2]
        depth_cropped_img = depth_img[(h-crop_1)//2:(h+crop_1)//2, (w-crop_2)//2-rand_1:(w+crop_2)//2+rand_2]

        rgb_resized_img = cv.resize(rgb_cropped_img, (240,240))
        depth_resized_img = cv.resize(depth_cropped_img, (240,240))
        
        new_name = file_name[:3] + str(ii) + file_name[3:] 
        
        rgb_path = os.path.join(target_rgb, new_name)
        depth_path = os.path.join(target_depth, new_name)

        ##### write ####
        cv.imwrite(rgb_path, rgb_resized_img)
        cv.imwrite(depth_path, depth_resized_img)

        angle = original_data.loc[original_data['Picture name'] == file_name[:-4], 'Angle [deg]'].values[0]
        radial = original_data.loc[original_data['Picture name'] == file_name[:-4], 'Radial distance [mm]'].values[0]
        cropped_data.append([new_name, angle, radial])

cropped_df = pd.DataFrame(cropped_data, columns=['Picture name', 'Angle [deg]', 'Radial distance [mm]'])
cropped_df.to_csv(csv_file_path, index=False)
    
    # ##### test #####
    # fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    # axes[0].imshow(rgb_resized_img)
    # axes[1].imshow(depth_resized_img)